### Evaluation of criticality metrics on the nuscenes dataset.

**Results produced for the nuscenes dataset are needed, as for example they can be produced by the notebook MMDetection3D, see the other notebook MMDetection3D**

**Note: nuscenes-dev must be correctly installed** We used version 1.1.2, but it has been tested up to version 1.1.7.

#### Some hints on the modifications done to the nuscenes library:

- detection/algo.py : the method "accumulate" computes precision, recall and average precision. The approach to compute precision and recall is as follows. NPOS=TP+FN is a static value and can be computed immediately. Then, an array with all the detection thresholds for tps is computed e.g. $[1\; 0\; 0\; 0\; 1]$. Same is done for fp. Same is done with the detection confidence. Then they are sorted according to the detection confidence, and elements are picked up from top to bottom. Precision and recall are computed with the retrieved items, iteratively. This is the equivalent of starting getting "detected items" with the highest confidence, and progressively get all the remaining items, and compute precision and recall at each step with all the items selected so far. To compute the precision curve, there is a marvellous interpolation of the precision matrix and the recall matrix with a predefined matrix.


- detection/evaluate.py : contains the main method


- detection/data_classes.py : DetectionBox, in the init method, it includes all the logic to perform computation of all criticalities for a target BoundingBox i.e., for each object. This is done independently if it is a ground truth bounding box or a predicted bounding box. So whenever we plan to change our geometry, we need to act here.


Further, image creation is manipulated so that it shows criticality values matched to each bounding box that is depicted.  It visualizes only one type of objects (CAR), for clarity of the image --> to change this, in utils/data_classes.py, render_crit, change the line "if(self.name!='car'):"

Programmatically, the base is visualize_sample_crit in detection/render.py; then there is method render_crit; and finally we build object Box in utils/data_classes.py; images are created at the beginning of main in detection/evaluate.py; the label is added in render_crit in utils/data_classes.py. To print a different class (e.g., car, bus, everything), should be easy working with the above files (the no_crit case prints everything is on map).

**Debug images** Debug images (folder examples_debug_1) are introduced. They consider only ground truth, and they print: velocity and position of all vehicles,including the ego vehicle. They are created as follows:
- in detection/evaluate.py there is method visualize_sample_debug_1 of eval.detection.render
- in eval.detection.render method visualize_sample_debug_1, for each boxes in gt, there is render_debug_1 of utils/data_classes.py
- utils/data_classes.py method render_debug_1 builds the squares of the different ground truth cars

#### Which results are computed

Results are saved in different folders. They are easy to find, you just need to configure the proper path as it will be explained a bit later here.


We collect results in terms of:

- precision recall curve, that we save as pdf files (see the pdf files with "crit" in the name)

We compute some more metrics in a text file, that is called "confusion matrix.txt". The file contains: i) average precision; ii) average precision crit;

*Example: class_name bus; dist_th 1.0; ap 0.16393238098925264; ap_crit 0.16604208258718856*

iii) MAXIMUM number of tp and fp; iv) MAXIMUM value of tp_crit pred and MAXIMUM value of tp_crit gt; iii) MAXIMUM number of tp; iv) MAXIMUM value of fp_crit pred; v) MINIMUM number of FN, and MINIMUM value of fn_crit gt

*Example: class_name car; dist_th 0.5; max tp 19758.0; max fp 177143.0; min fn 33285.0; max tp_pred_crit 19249.891210080583; max tp_gt_crit 19272.72239514313; max fp_pred_crit 172455.20687223336; min fn_gt_crit 32468.569145629514*

To enrich this with multiple metrics, just operate on method *accumulate* in *algo.py*

Last, you can save bird eye view images that have the crit values added to each represented bounding box.

ACRONYMS (as naming in the following differs with respect to the paper)

First column is the algorithm name (as close as possible to naming in mmdetection3d), second is the abbreviation we use in our paper

- FCOSD-RESNET101              FCOS
- pointpillars-secfpn          SEC
- pointpillars-fpn             FPN
- regnet-regnetX_400MF-FPN     REG400
- ssn-SECFPN                   SSN
- regnet-regnetX_400MF-SECFPN  REGSEC (REG400SEC)
- ssn-REGNET                   SSNREG
- regnet-regnetX_FPN           REG1.6
- pgd                          PGD


Also, we have:

- D in the paper is:    $D_{max}$
- T in the paper is:        $T_{max}$
- I in the paper is:        $R_{max}$

*reminder:* if you ever need to change dist_ths, dist_thp: check configs/detection_cvpr_2019.json

#### Below are configuration items

In [2]:


## POINTPILLARS WITH FPN BACKBONE -- lidar only
#PATH='/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/results/pointpillars_nuscenes_results/pts_bbox/'
#FILE_JSON='results_nusc.json'
#ETECTOR='pointpillars-fpn'

##RegNET WITH BACKBONE RegNetX-1.6gF-FPN
PATH='/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/results/regnetX_nuscenes_results/pts_bbox/' 
FILE_JSON='results_nusc.json'
DETECTOR='regnet-regnetX_FPN'

#### SSN con backbone REGNET 400MF SECFPN  -- LIDAR-only
#PATH='/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/results/ssn_nuscenes_results/pts_bbox/' 
#FILE_JSON='results_nusc.json'
#DETECTOR='ssn-REGNET'


#nuscene dataroot
DATAROOT = '/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/data/nuscenes'
#path + json file where detection results from mmdetection3d are stored, ready to be processed

RESULT_PATH=PATH+FILE_JSON
#the detector in use. Does nothing special but creates a folder, and it is useful to put results there
#results of evaluation will be stored here
OUTPUT='/cluster/work/andronn/MasterThesis/MASTER/CriticalityModel/results/'+DETECTOR+"/"

#parameters from our marvellous solution. At the bottom of the notebook, there is a different 
#approach where multiple config are tested
MAX_DISTANCE_OBJ_1=30.0
MAX_DISTANCE_INTERSECT_1=20.0
MAX_TIME_INTERSECT_OBJ_1=10.0

#how many images in bird view you want to draw
NUMBER_IMAGE=0

In [3]:
#required if you are playing with libraries and changing them. This reloads libraries
%load_ext autoreload
%autoreload 2

In [4]:
import unittest
import numpy as np
import sklearn
import tqdm
import pandas
import math
import json
from typing import Callable
from nuscenes import NuScenes
from nuscenes.eval.prediction.splits import *
import nuscenes.eval.detection.config as cnfig
from nuscenes.eval.detection.configs import *
from nuscenes.eval.detection.data_classes import DetectionBox 
from nuscenes.eval.detection import *
import nuscenes.eval.detection.algo as ag
from nuscenes.eval.detection.data_classes import DetectionMetricData, DetectionConfig, DetectionMetrics, DetectionBox, \
    DetectionMetricDataList
from nuscenes.eval.common.data_classes import EvalBoxes

import os
from typing import List, Dict, Callable, Tuple
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean
import nuscenes.eval.detection.evaluate as dcl    
from nuscenes.prediction import *
from nuscenes.map_expansion.map_api import *

In [5]:
nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 45.031 seconds.
Reverse indexing ...
Done reverse indexing in 11.9 seconds.


In [6]:
confvalue=cnfig.config_factory("detection_cvpr_2019")

In [7]:
#number of scene that compose the val set
eval=val = \
    ['scene-0003', 'scene-0012', 'scene-0013', 'scene-0014', 'scene-0015', 'scene-0016', 'scene-0017', 'scene-0018',
     'scene-0035', 'scene-0036', 'scene-0038', 'scene-0039', 'scene-0092', 'scene-0093', 'scene-0094', 'scene-0095',
     'scene-0096', 'scene-0097', 'scene-0098', 'scene-0099', 'scene-0100', 'scene-0101', 'scene-0102', 'scene-0103',
     'scene-0104', 'scene-0105', 'scene-0106', 'scene-0107', 'scene-0108', 'scene-0109', 'scene-0110', 'scene-0221',
     'scene-0268', 'scene-0269', 'scene-0270', 'scene-0271', 'scene-0272', 'scene-0273', 'scene-0274', 'scene-0275',
     'scene-0276', 'scene-0277', 'scene-0278', 'scene-0329', 'scene-0330', 'scene-0331', 'scene-0332', 'scene-0344',
     'scene-0345', 'scene-0346', 'scene-0519', 'scene-0520', 'scene-0521', 'scene-0522', 'scene-0523', 'scene-0524',
     'scene-0552', 'scene-0553', 'scene-0554', 'scene-0555', 'scene-0556', 'scene-0557', 'scene-0558', 'scene-0559',
     'scene-0560', 'scene-0561', 'scene-0562', 'scene-0563', 'scene-0564', 'scene-0565', 'scene-0625', 'scene-0626',
     'scene-0627', 'scene-0629', 'scene-0630', 'scene-0632', 'scene-0633', 'scene-0634', 'scene-0635', 'scene-0636',
     'scene-0637', 'scene-0638', 'scene-0770', 'scene-0771', 'scene-0775', 'scene-0777', 'scene-0778', 'scene-0780',
     'scene-0781', 'scene-0782', 'scene-0783', 'scene-0784', 'scene-0794', 'scene-0795', 'scene-0796', 'scene-0797',
     'scene-0798', 'scene-0799', 'scene-0800', 'scene-0802', 'scene-0904', 'scene-0905', 'scene-0906', 'scene-0907',
     'scene-0908', 'scene-0909', 'scene-0910', 'scene-0911', 'scene-0912', 'scene-0913', 'scene-0914', 'scene-0915',
     'scene-0916', 'scene-0917', 'scene-0919', 'scene-0920', 'scene-0921', 'scene-0922', 'scene-0923', 'scene-0924',
     'scene-0925', 'scene-0926', 'scene-0927', 'scene-0928', 'scene-0929', 'scene-0930', 'scene-0931', 'scene-0962',
     'scene-0963', 'scene-0966', 'scene-0967', 'scene-0968', 'scene-0969', 'scene-0971', 'scene-0972', 'scene-1059',
     'scene-1060', 'scene-1061', 'scene-1062', 'scene-1063', 'scene-1064', 'scene-1065', 'scene-1066', 'scene-1067',
     'scene-1068', 'scene-1069', 'scene-1070', 'scene-1071', 'scene-1072', 'scene-1073']

In [8]:
len(val)

150

In [16]:
%reload_ext autoreload
dt=dcl.DetectionEval(nuscenes,confvalue, RESULT_PATH, 'val', OUTPUT, verbose=False, MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
                     MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1, MAX_TIME_INTERSECT_OBJ=MAX_TIME_INTERSECT_OBJ_1)

### Calculate PKLs
Using planning_centric_metrics, calculate PKL for 

In [ ]:
from nuscenes.map_expansion.map_api import NuScenesMap
from planning_centric_metrics import calculate_pkl
import torch


device = torch.device(torch.device('cpu'))
print('using device: {0}'.format(device))

map_folder = '/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/data/nuscenes/maps/'
nusc_maps = {map_name: NuScenesMap(dataroot=map_folder,
                 map_name=map_name) for map_name in [
                    "singapore-hollandvillage",
                    "singapore-queenstown",
                    "boston-seaport",
                    "singapore-onenorth",
                ]}
nworkers = 1

pkl = calculate_pkl(dt.gt_boxes, dt.pred_boxes,
                         dt.sample_tokens, dt.nusc,
                         nusc_maps, device,
                         nworkers=nworkers, bsz=128,
                         plot_kextremes=0,
                         verbose=True)
with open(os.path.join(OUTPUT,'pkl_results.json'), 'w') as fp:
    json.dump(pkl, fp)


In [17]:
%reload_ext autoreload

#dt.main(plot_examples=NUMBER_IMAGE,render_curves=True)


dt.main(plot_examples=NUMBER_IMAGE,
        render_curves=True, 
        model_name=DETECTOR,
        MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
        MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1,
        MAX_TIME_INTERSECT=MAX_TIME_INTERSECT_OBJ_1,
        recall_type="PRED AL NUMERATORE") #This must be "PRED AL NUMERATORE", to match results of the paper


STARTING EVALUATION in main (self)
STARTING EVALUATION in evaluate(self)


{'label_aps': defaultdict(<function nuscenes.eval.detection.data_classes.DetectionMetrics.__init__.<locals>.<lambda>()>,
             {'car': defaultdict(float,
                          {0.5: 0.7009041224029755,
                           1.0: 0.8232151604547453,
                           2.0: 0.8510441773941825,
                           4.0: 0.8669954356972513}),
              'truck': defaultdict(float,
                          {0.5: 0.2396241959258286,
                           1.0: 0.4405943521053402,
                           2.0: 0.5518397942462654,
                           4.0: 0.6031007244549175}),
              'bus': defaultdict(float,
                          {0.5: 0.1742220844586778,
                           1.0: 0.5537224010560102,
                           2.0: 0.7173231905301807,
                           4.0: 0.7466024870609486}),
              'trailer': defaultdict(float,
                          {0.5: 0.013489289696473606,
                           1.

### This is automatic, to test multiple settings

We can compute the $AP_{crit}$ and related metrics considering different values of D, T, I. This is in fact what is done in the paper, where 1500 different triples of D, I, T are considered. The default settings (the one in the paper) are below. Note that it will take a couple of days to complete. 

You may want to test a smaller number of triples D, T, I: just change the "combined_list" values.

In [ ]:
"""VARIOUS_DISTANCE_OBJ_ARRAY=[]
VARIOUS_DISTANCE_OBJ_ARRAY.extend(range(5, 51, 5))
VARIOUS_DISTANCE_INTERSECT_ARRAY=[]
VARIOUS_DISTANCE_INTERSECT_ARRAY.extend(range(5, 51, 5))
VARIOUS_TIME_INTERSECT_ARRAY=[]
VARIOUS_TIME_INTERSECT_ARRAY.extend(range(2,31,2))

import itertools
a = [VARIOUS_DISTANCE_OBJ_ARRAY,VARIOUS_DISTANCE_INTERSECT_ARRAY,VARIOUS_TIME_INTERSECT_ARRAY]
combined_list=list(itertools.product(*a))"""

In [ ]:
#how many images in bird view you want to draw
"""NUMBER_IMAGE=0
rendering=False
len(combined_list)"""

In [ ]:
"""def compute_values(PATH, FILE_JSON, DETECTOR, RESULT_PATH, OUTPUT):
    for j in range(len(combined_list)):
        OUTPUT1=OUTPUT+'/'+'DATA'+'/iteration_'+str(j)+'/'

        dt=dcl.DetectionEval(nuscenes,
                             confvalue, RESULT_PATH, 'val', OUTPUT1, verbose=False,
                             MAX_DISTANCE_OBJ=combined_list[j][0],
                             MAX_DISTANCE_INTERSECT=combined_list[j][1], 
                             MAX_TIME_INTERSECT_OBJ=combined_list[j][2])
    #    dt.main(plot_examples=NUMBER_IMAGE,render_curves=True)
        dt.main(plot_examples=NUMBER_IMAGE,
                render_curves=rendering, 
                model_name=DETECTOR,
                MAX_DISTANCE_OBJ=combined_list[j][0],
                MAX_DISTANCE_INTERSECT=combined_list[j][1],
                MAX_TIME_INTERSECT=combined_list[j][2],
                recall_type="PRED AL NUMERATORE") 
        f = open(OUTPUT1+"README_CONFIG.txt", "w")
        f.write("Configurations for this iteration\n")
        f.write("RECALL TYPE: CRIT (AS THE PAPER)\n")
        f.write("MAX_DISTANCE_OBJ "+str(combined_list[j][0])+"\n")
        f.write("MAX_DISTANCE_INTERSECT "+str(combined_list[j][1])+"\n")
        f.write("MAX_TIME_INTERSECT "+str(combined_list[j][2])+"\n")
        f.close()
        del dt"""


In [ ]:
"""PATH='/home/notebook/mmdetection3d/pgd_results/img_bbox/'
FILE_JSON='results_nusc.json'
DETECTOR='pgd'

RESULT_PATH=PATH+FILE_JSON
OUTPUT='/home/notebook/results/'+DETECTOR+"/""""

In [ ]:
"""%%capture
%reload_ext autoreload
compute_values(PATH, FILE_JSON, DETECTOR, RESULT_PATH, OUTPUT);
"""